In [1]:
import pandas as pd
import numpy as np

In [2]:
link1 = "./BD/df_basic_name_clean.csv"
link2 = "./BD/df_t_akas_clean.csv"
link3 = "./BD/df_t_basics_clean.csv"
link4 = "./BD/df_t_crew_clean.csv"
link5 = "./BD/df_t_episode_clean.csv"
link6 = "./BD/df_t_principals_clean.csv"
link7 = "./BD/df_t_ratings_clean.csv"
link8 = "./BD/tmdb_full.csv"
link9 = "./BD/df_tID.csv"
link10 = "./BD/df_nID.csv"
link11 = "./BD/df_tmdb_clean.csv"

In [11]:
df_basic_name = pd.read_csv(link1)

In [ ]:
df_t_akas = pd.read_csv(link2)

In [3]:
df_t_basics = pd.read_csv(link3)

In [ ]:
df_t_crew = pd.read_csv(link4)

In [ ]:
df_t_episode = pd.read_csv(link5)

In [8]:
df_t_principals = pd.read_csv(link6)

In [ ]:

df_t_ratings = pd.read_csv(link7)

In [58]:
#Petite fonction pour créer une colonne période si on a l'année:
def periode(x):
    if x < 1900:
        return "Années 1800"
    elif 1900 <= x < 1910:
        return 'Années 1900' 
    elif 1910 <= x < 1920:
        return "Années 1910"
    elif 1920 <= x < 1930:
        return "Années 1920"
    elif 1930 <= x < 1940:
        return "Années 1930"
    elif 1940 <= x < 1950:
        return "Années 1940"
    elif 1950 <= x < 1960:
        return "Années 1950"
    elif 1960 <= x < 1970:
        return "Années 1960"
    elif 1970 <= x < 1980:
        return "Années 1970"
    elif 1980 <= x < 1990:
        return "Années 1980"
    elif 1990 <= x < 2000:
        return "Années 1990"
    elif 2000 <= x < 2010:
        return "Années 2000"
    elif 2010 <= x < 2020:
        return "Années 2010"
    else:
        return "Années 2020"

# KPI1 Identification des acteurs les plus présents et les périodes associées

In [ ]:
# Prépartion table title basics: On va garder les ID de films qui correspondent à des films en enlevant les pornos et les startYear NaN:
df_tb_tconst = df_t_basics[(df_t_basics['titleType'] == 'movie')  & (df_t_basics['isAdult'] == 0) & (df_t_basics['startYear'].isna() == False)]
df_tb_tconst = df_tb_tconst[['tconst', 'startYear']]

#Préparation table title principals pour afficher des listes d'acteurs pour chaque id de film:
df_ptconst = df_t_principals[(df_t_principals['category'] == 'actor') | (df_t_principals['category'] == 'actress')]
df_ptconst = df_ptconst.drop('category', axis = 1)
df_ptconst = df_ptconst .groupby('tconst').agg(list)

#Premier merge de title principal sur les tconst triés de title basic:
df_KPI1 = pd.merge(df_tb_tconst, df_ptconst, how = 'left', on='tconst')

#Certains films n'ont pas de liste d'acteur correspondants, on va pouvoir les dropper:
df_KPI1 = df_KPI1.dropna()


#Incorporation des noms des acteurs:
#On utilise la table basic name qui associe un ID a un nom et on les réunis en dico {ID : nom}:
dico = dict(zip(df_basic_name["nconst"], df_basic_name["primaryName"]))

#On défini une fonction pour faire ce que l'on cherche et on applique:
def id_nom(x):
    rendu = []
    for id in x:
        if id in dico.keys(): #Il y a quelques id inconnus je suis obligé de mettre une condition
            rendu.append(dico[id])
    return rendu

df_KPI1["Nom_acteurs"] = df_KPI1['nconst'].apply(id_nom)

#Plus besoin de nconst:
df_KPI1= df_KPI1.drop('nconst', axis = 1)

#On explode tout ça:
df_KPI1 = df_KPI1.explode('Nom_acteurs')

#On applique la fonction période du dessus:
df_KPI1['décénie'] = df_KPI1['startYear'].apply(periode)

#Plus joli avec l'année en int:
df_KPI1['startYear'] = df_KPI1['startYear'].astype('int')

#reset index c'est toujours plus propre:
df_KPI1= df_KPI1.reset_index().drop('index', axis = 1)

#Export pour powerBI:
df_KPI1.to_csv('df_KPI1.csv', index = False)

In [68]:
df_KPI1

,tconst,startYear,Nom_acteurs,décénie
0,tt0000009,1894,Blanche Bayliss,Années 1800
1,tt0000009,1894,William Courtenay,Années 1800
2,tt0000009,1894,Chauncey Depew,Années 1800
3,tt0000502,1905,Antonio del Pozo,Années 1900
4,tt0000502,1905,El Mochuelo,Années 1900
...,...,...,...,...
3520346,tt9916730,2017,Bhushan Pradhan,Années 2010
3520347,tt9916730,2017,Pranav Raorane,Années 2010
3520348,tt9916730,2017,Aarti Solanki,Années 2010
3520349,tt9916730,2017,Amruta Subhash,Années 2010


# KPI2 : Evolution de la durée moyenne des films au fil des années

In [ ]:
# Prépartion table title basics: On va garder les ID de films qui correspondent à des films en enlevant les pornos et les startYear NaN:
df_tb_tconst = df_t_basics[(df_t_basics['titleType'] == 'movie')  & (df_t_basics['isAdult'] == 0) & (df_t_basics['startYear'].isna() == False)]
df_tb_tconst = df_tb_tconst[['tconst', 'startYear', 'runtimeMinutes']]

#On élimine les runtimes non renseignés:
df_KPI2 = df_tb_tconst.dropna()

#Plus joli avec l'année en int:
df_KPI2['startYear'] = df_KPI2['startYear'].astype('int')

#On applique la fonction periode:
df_KPI2['décénie'] = df_KPI2['startYear'].apply(periode)

#Toujours mon jÔli petit index:
df_KPI2 = df_KPI2.reset_index().drop('index', axis = 1)

#Et on peut exporter!
df_KPI2.to_csv('df_KPI2.csv', index = False)


In [71]:
df_KPI2

,tconst,startYear,runtimeMinutes,décénie
0,tt0000009,1894,45.0,Années 1800
1,tt0000147,1897,100.0,Années 1800
2,tt0000502,1905,100.0,Années 1900
3,tt0000574,1906,70.0,Années 1900
4,tt0000591,1907,90.0,Années 1900
...,...,...,...,...
426245,tt9916538,2019,123.0,Années 2010
426246,tt9916622,2015,57.0,Années 2010
426247,tt9916680,2007,100.0,Années 2000
426248,tt9916730,2017,116.0,Années 2010


# KPI3 : Comparaison entre les acteurs présents au cinéma et dans les séries

# KPI4 : Age moyen des acteurs.

# KPI5 : Les films les mieux notés et les caractéristiques qu’ils partagent.